In [1]:
import qsharp
import math

n = 5 # amount of qubits
m = 1 # number of marked items
optimalIterations = (math.pi / 4) * math.sqrt(math.pow(2, n)/m)
optimalIterations = math.floor(optimalIterations)

print("Optimal number of iterations for Grover's algorithm with {} qubits and {} marked items: {}".format(n, m, optimalIterations))

Optimal number of iterations for Grover's algorithm with 5 qubits and 1 marked items: 4


In [2]:
%%qsharp
import Microsoft.Quantum.Convert.*;
import Microsoft.Quantum.Math.*;
import Microsoft.Quantum.Arrays.*;
import Microsoft.Quantum.Measurement.*;
import Microsoft.Quantum.Diagnostics.*;

operation Run(nQubits: Int, iterations: Int) : String {
    Message($"Running Grover's algorithm with {nQubits} qubits and {iterations} iterations...");

    let results = GroverSearch(nQubits, iterations, CustomOracle);
    return ResultArrayToString(results);
}

operation GroverSearch(
    nQubits : Int,
    iterations : Int,
    phaseOracle : Qubit[] => Unit
) : Result[] {

    use qubits = Qubit[nQubits];

    PrepareUniform(qubits);

    for _ in 1..iterations {
        phaseOracle(qubits);
        ReflectAboutUniform(qubits);
    }

    // Measure and return the answer.
    return MResetEachZ(qubits);
}

operation CustomOracle(inputQubits : Qubit[]) : Unit {
    Message("Reflecting about marked state...");
    use outputQubit = Qubit();
    within {
        X(outputQubit);
        H(outputQubit);
        for q in inputQubits[...2...] {
            X(q);
        }
    } apply {
        Controlled X(inputQubits, outputQubit);
    }
}

operation PrepareUniform(inputQubits : Qubit[]) : Unit is Adj + Ctl {
    for q in inputQubits {
        H(q);
    }
}

operation ReflectAboutAllOnes(inputQubits : Qubit[]) : Unit {
    Controlled Z(Most(inputQubits), Tail(inputQubits));
}

operation ReflectAboutUniform(inputQubits : Qubit[]) : Unit {
    within {
        Adjoint PrepareUniform(inputQubits);
        for q in inputQubits {
            X(q);
        }
    } apply {
        ReflectAboutAllOnes(inputQubits);
    }
}

function ResultArrayToString(results : Result[]) : String {
    mutable str = "";
    for r in results {
        set str += r == One ? "1" | "0";
    }
    return str;
}

Run(5, 4);

Running Grover's algorithm with 5 qubits and 4 iterations...

Reflecting about marked state...

Reflecting about marked state...

Reflecting about marked state...

Reflecting about marked state...

'01010'

In [ ]:
results = qsharp.run("Run", n, optimalIterations)
print(results)

<Global> <item 0 in package 3>
